In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [7]:
from pathlib import Path
import json

In [8]:
with open('../retriever/dataset/split.json', 'r') as file:
    split = json.load(file)
with open('../topics.json', 'r') as file:
    topics = json.load(file)

In [9]:
ref = {'false': 0, 'on_fire': 0, 'mostly_false': 1, 'half_true': 2, 'mostly_true': 3, 'true': 4}

In [10]:
for i in range(5):
    in_cnt = 0
    in_sum = 0
    out_cnt = 0
    out_sum = 0
    output = {}
    path = Path(f'./model/{i+1}')
    sub = path.glob('*')
    model = AutoModelForSequenceClassification.from_pretrained(sorted(sub)[-1])
    for topic in topics.keys():
        print(topic)
        for label in ref.keys():
            hit = 0
            total = 0
            path = Path(f'../retrieval/{i+1}/test/{topic}/{label}')
            sub = path.glob('*')
            num = len(list(sub))
            for j in range(num):
                idx = j+1
                path = Path(f'../retrieval/{i+1}/test/{topic}/{label}/{idx}')
                sub = path.glob('*')
                with open(f'../dataset/test/{topic}/{label}/{idx}/info.json', 'r') as file:
                    info = json.load(file)
                claim = info['claim']
                cnt = {'false': 0, 'mostly_false': 0, 'half_true': 0, 'mostly_true': 0, 'true': 0}
                for s in sub:
                    with open(s, 'r') as file:
                        evidence = json.load(file)
                    text = ""
                    for e in evidence:
                        text += str(e) + ' '
                    input = f'Claim: {claim}\nReference: {text}'
                    inputs = tokenizer(input, return_tensors="pt", truncation=True)
                    with torch.no_grad():
                        logits = model(**inputs).logits
                    predicted_class_id = logits.argmax().item()
                    score = logits[0][predicted_class_id]
                    predicted_class = model.config.id2label[predicted_class_id]
                    if score >= 0.25:
                        cnt[predicted_class] += 1
                sorted_cnt = dict(sorted(cnt.items(), key=lambda item: item[1], reverse=True))
                prediction = list(sorted_cnt.keys())[0]
                if label == 'on_fire':
                    if prediction == 'false':
                        prediction = 'on_fire'
                total += 1
                if prediction == label:
                    hit += 1
                if topic in split[i]:
                    in_sum += 1
                    if prediction == label:
                        in_cnt += 1
                else:
                    out_sum += 1
                    if prediction == label:
                        out_cnt += 1
            
            out = {'hit': hit, 'total': total, 'rate': hit/total}
            if topic not in output:
                output[topic] = {}
            output[topic][label] = out
    with open(f'../retrieve-baseline-{i+1}.json', 'w') as file:
        json.dump(output, file)
    # print(f'Group {i+1} - In group: {in_cnt}/{in_sum}, Out group: {out_cnt}/{out_sum}')

Energy
Jobs
Legal Issues
Economy
Taxes
Technology
Education
Health Care
Immigration
Foreign Policy
LGBTQ
Military
Elections
Environment
Transportation
Terrorism
Sports
State Budget
Federal Budget
Criminal Justice
Religion
Government Regulation
Candidate Biography
Homeland Security
Social Security
History
Group 5 - In group: 220/664, Out group: 763/2560
